# makes the 'big trackhub' for the washU Genomebrowser
- the important stuff is that it subsets each overlapping exon

In [1]:
! cd /home/bay001/projects/codebase/bfx/;python setup.py build && python setup.py install

/home/bay001/anaconda/lib/python2.7/distutils/dist.py:267: UserWarning: Unknown distribution option: 'include_package_data'
  warnings.warn(msg)
/home/bay001/anaconda/lib/python2.7/distutils/dist.py:267: UserWarning: Unknown distribution option: 'install_requires'
  warnings.warn(msg)
running build
running build_py
/home/bay001/anaconda/lib/python2.7/distutils/dist.py:267: UserWarning: Unknown distribution option: 'include_package_data'
  warnings.warn(msg)
/home/bay001/anaconda/lib/python2.7/distutils/dist.py:267: UserWarning: Unknown distribution option: 'install_requires'
  warnings.warn(msg)
running install
running build
running build_py
running install_lib
running install_egg_info
Removing /home/bay001/anaconda/lib/python2.7/site-packages/bfx-0.0.0-py2.7.egg-info
Writing /home/bay001/anaconda/lib/python2.7/site-packages/bfx-0.0.0-py2.7.egg-info


In [131]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from rnaseq import rmats_inclevel_analysis as rmats
from encode import manifest_helpers
import pandas as pd
import numpy as np
import os
import glob
from tqdm import tnrange, tqdm_notebook
import pybedtools

pd.set_option('display.max_columns', 500)


## make merged incleveldifference matrix
- merges junction regions and get average incleveldifference for each RBP
- performs outer join on junction regions
- returns full matrix of RBPs

/home/bay001/anaconda/lib/python2.7/site-packages/rnaseq/rmats_inclevel_analysis.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['index'] = df.apply(event_to_string, axis=1)


In [132]:
help(manifest_helpers)

Help on module encode.manifest_helpers in encode:

NAME
    encode.manifest_helpers - Created on Feb 2, 2017

FILE
    /home/bay001/anaconda/lib/python2.7/site-packages/encode/manifest_helpers.py

DESCRIPTION
    Customized script for parsing yeolab ENCODE-style manifests. 
    See data/ for examples
    @author: brianyee

FUNCTIONS
    get_all_info_for_one_expt(uid, annotation_dir, clip_manifest_df, rnaseq_manifests)
        Parameters
        ----------
        uid : basestring
            uID of a given clip expt
        annotation_dir : basestring
            directory where all the annotations downloaded from graveleylab are
        clip_manifest_df : pandas.DataFrame
            manifest file containing all submitted clips
            associates RBP -> CLIP expt
        rnaseq_manifests : basestring
            manifest file containing all submitted kd expts
            associates RBP -> shRNA KD splicing rmats expt
        
        Returns
        -------
    
    get_annotation

# get_average_of_overlapping_cassettes
- merge bedtool
- get intersection between bedfile and each nonoverlapping region from merge
- break each intersected group into parts using intersect and groupby
- overlapping regions will have the average dpsi value of the overlaps
- nonoverlapping regions will retain their dpsi value

# Test functions
- these aren't really bedtools, these all return dataframes because they're visible. 

In [5]:
def fake_completely_overlapping_bedtool():
    intervals = []
    intervals.append(pybedtools.create_interval_from_list([
                'chr1', '500', '2500', 'big', '0.75', '+'
            ]))
    intervals.append(pybedtools.create_interval_from_list([
                'chr1', '1000', '2000', 'small', '0.5', '+'
            ]))
    bedtool = pybedtools.BedTool(intervals).to_dataframe()
    bedtool.columns = ['chr','exonStart_0base','exonEnd','geneSymbol','IncLevelDifference','strand']
    return bedtool

def fake_partially_overlapping_bedtool():
    intervals = []
    intervals.append(pybedtools.create_interval_from_list([
                'chr1', '500', '1000', 'genex', '0.75', '+'
            ]))
    intervals.append(pybedtools.create_interval_from_list([
                'chr1', '750', '1500', 'genex', '0.5', '+'
            ]))
    bedtool = pybedtools.BedTool(intervals).to_dataframe()
    bedtool.columns = ['chr','exonStart_0base','exonEnd','geneSymbol','IncLevelDifference','strand']
    return bedtool

def fake_share_start_exon_bedtool():
    intervals = []
    intervals.append(pybedtools.create_interval_from_list([
                'chr1', '500', '1000', 'genex', '0.75', '+'
            ]))
    intervals.append(pybedtools.create_interval_from_list([
                'chr1', '500', '1500', 'genex', '0.5', '+'
            ]))
    bedtool = pybedtools.BedTool(intervals).to_dataframe()
    bedtool.columns = ['chr','exonStart_0base','exonEnd','geneSymbol','IncLevelDifference','strand']
    return bedtool

def fake_share_end_exon_bedtool():
    intervals = []
    intervals.append(pybedtools.create_interval_from_list([
                'chr1', '500', '1500', 'genex', '0.75', '+'
            ]))
    intervals.append(pybedtools.create_interval_from_list([
                'chr1', '750', '1500', 'genex', '0.5', '+'
            ]))
    bedtool = pybedtools.BedTool(intervals).to_dataframe()
    bedtool.columns = ['chr','exonStart_0base','exonEnd','geneSymbol','IncLevelDifference','strand']
    return bedtool

# The actual functions:

In [30]:
def make_rmats_bedtool(df):
    """
    Uses the skipped exon start and end to create a bedtool
    """
    df = df[['chr','exonStart_0base','exonEnd','geneSymbol','IncLevelDifference','strand']]
    bt = pybedtools.BedTool.from_dataframe(df)
    bt = bt.sort()
    return bt

def make_bedtool(df):
    """
    I can't figure out why the BedTool() function isn't working...
    Probably has something to do with turning positions into floats,
    but this function is works just the same...
    """
    intervals = []
    
    for col, row in df.iterrows():
        intervals.append(
            pybedtools.create_interval_from_list(
                [str(row['chrom']), str(row['start']), 
                 str(row['end']), str(row['name']), 
                 str(row['score']), str(row['strand'])]
            )
        )
    return pybedtools.BedTool(intervals)

def redefine_regions(df):
    """
    turns overlapping regions into distinct nonoverlapping regions.
    """
    positions = []
    intervals = []
    for col, row in df.iterrows():
        chrom = row['chrom']
        strand = row['strand']
        positions.append(row['start'])
        positions.append(row['end'])
    positions = sorted(set(positions))
    for p in range(0, len(positions[:-1])):
        intervals.append(pybedtools.create_interval_from_list(
            [chrom, str(positions[p]), str(positions[p+1]), 'name', '0', strand]
        ))
    return pybedtools.BedTool(intervals)

def rescore(to_split):
    """
    Takes a dataframe of overlapping intervals, 
    and returns nonoverlapping regions, scored by 
    either taking the average of the original overlapping region,
    or by taking the single score over the nonoverlapping
    regions. 
    """
    name = to_split['name'].value_counts()[0] # just take the first name, i don't really care about the name part anyway
    final_split = pd.DataFrame(
        make_bedtool(to_split).intersect(
            redefine_regions(to_split)).to_dataframe().groupby(
            ['chrom','start','end','strand'])['score'].mean()
        ).reset_index()
    final_split['name'] = name
    final_split = final_split[['chrom','start','end','name','score','strand']]
    return final_split

def full_wrapper(df):
    """
    Takes a dataframe from an RMATS file and turns it into a BedTool.
    
    Calls 'pybedtools.cluster().to_dataframe()', which groups overlapping 
    regions using the 'thickStart' column.
    
    For each group, if there is only one region within the group, do nothing
    (concat to merged). If there is more than one region, this means we have
    overlapping intervals. Then it must call rescore() to split these regions
    into nonoverlapping intervals. 
    """
    dfx = make_rmats_bedtool(df)
    dfy = dfx.cluster().to_dataframe()
    merged = pd.DataFrame(columns=['chrom','start','end','name','score','strand','thickStart'])
    groups = set(dfy['thickStart'])
    progress = tnrange(len(groups))
    for g in groups:
        dft = dfy[dfy['thickStart']==g] # get all overlapping regions
        if dft.shape[0] > 1:
            merged = pd.concat([merged, rescore(dft)])
        else:
            merged = pd.concat([merged, dft])
        progress.update(1)
    merged = merged[['chrom','start','end','name','score','strand']]
    return merged

# Test functions

# region1 encompasses region2 fully

In [7]:
df = fake_completely_overlapping_bedtool()
df

,chr,exonStart_0base,exonEnd,geneSymbol,IncLevelDifference,strand
0,chr1,500,2500,big,0.75,+
1,chr1,1000,2000,small,0.50,+


In [8]:
full_wrapper(df)

,chrom,start,end,name,score,strand
0,chr1,500.0,1000.0,big,0.750,+
1,chr1,1000.0,2000.0,big,0.625,+
2,chr1,2000.0,2500.0,big,0.750,+


# exon1 start < exon2 start, exon1 end < exon2 end

In [9]:
df = fake_partially_overlapping_bedtool()
df

,chr,exonStart_0base,exonEnd,geneSymbol,IncLevelDifference,strand
0,chr1,500,1000,genex,0.75,+
1,chr1,750,1500,genex,0.50,+


In [10]:
full_wrapper(df)

,chrom,start,end,name,score,strand
0,chr1,500.0,750.0,genex,0.750,+
1,chr1,750.0,1000.0,genex,0.625,+
2,chr1,1000.0,1500.0,genex,0.500,+


# Share start exon

In [11]:
df = fake_share_start_exon_bedtool()
df

,chr,exonStart_0base,exonEnd,geneSymbol,IncLevelDifference,strand
0,chr1,500,1000,genex,0.75,+
1,chr1,500,1500,genex,0.50,+


In [12]:
full_wrapper(df)

,chrom,start,end,name,score,strand
0,chr1,500.0,1000.0,genex,0.625,+
1,chr1,1000.0,1500.0,genex,0.500,+


# Share end exon

In [13]:
df = fake_share_end_exon_bedtool()
df

,chr,exonStart_0base,exonEnd,geneSymbol,IncLevelDifference,strand
0,chr1,500,1500,genex,0.75,+
1,chr1,750,1500,genex,0.50,+


In [14]:
full_wrapper(df)

,chrom,start,end,name,score,strand
0,chr1,500.0,750.0,genex,0.750,+
1,chr1,750.0,1500.0,genex,0.625,+


# Now loop over all RMATS files and return nonoverlapping scores for each RBP

In [32]:
def loop_over_rmats_filtered_files():
    values = []
    for rbp, value in dfx.iteritems():
        values.append(value)
    return values[:2]

In [97]:
y, x = loop_over_rmats_filtered_files()

In [98]:
x = full_wrapper(x)
# y = full_wrapper(y)

In [129]:
y = full_wrapper(y)

In [100]:
# playing around with the column stuff
x.columns = ['chrom','start','end','name',234,'strand']
x[235] = x[234] + 3
trunc = x.head()

In [162]:
from collections import defaultdict

def format_df_to_trackhub(df, out_file):
    """
    kind of a messy way to re-format the dataframe
    """
    o = open(out_file, 'w')
    for _, row in df.iterrows():
        row_str = "{}\t{}\t{}\t".format(row['chrom'], row['start'], row['end'])
        row_str = row_str + 'id:{},qcat:'.format(_)
        qcat_str = ""
        for i in row.index:
            if isinstance(i, int) and not pd.isnull(row[i]):
                qcat_str = qcat_str + '[{},{}], '.format(i, row[i])
        qcat_str = qcat_str[:-2]
        o.write(row_str + '[ {} ]\n'.format(qcat_str))
    o.close()
    return 0

def rbp_to_qcat(json_like):
    """
    turns this json like file into a dictionary
    with rbp names as keys and category ID, color as values
    """
    categories = defaultdict(list)
    with open(json_like, 'r') as f:
        for line in f:
            if line.startswith('\t'):
                try:
                    line = line.replace('\'','')
                    category, rbp = line.replace('[','').replace(']','').split(':')
                    rbpname, rbpcolor, _ = rbp.split(',')
                    categories[rbpname] = [int(category.replace('\t','')), rbpcolor]
                except ValueError:
                    print(line)
    return categories

json_file = '/home/bay001/projects/encode/analysis/rnaseq_trackhub/combined_10bpfull.datahub.pos'
qcat_dict = rbp_to_qcat(json_file)
# format_df_to_trackhub(trunc, '/home/bay001/test.bed') # .to_csv('/home/bay001/test.bed', sep='\t', header=False)

	},



In [154]:
import json
from pprint import pprint
# Can't parse this file as json...
"""json_file = '/home/bay001/projects/encode/analysis/rnaseq_trackhub/combined_10bpfull.datahub.pos'
with open(json_file) as data_file:    
    data = json.load(data_file)"""

"json_file = '/home/bay001/projects/encode/analysis/rnaseq_trackhub/combined_10bpfull.datahub.pos'\nwith open(json_file) as data_file:    \n    data = json.load(data_file)"

In [158]:
def get_all_junction_counts_files(wd):
    """
    gets all junctioncounts (SE) from a directory (wd)
    """
    all_junction_counts_dict = {}
    for cell in ['K562','HepG2']:
        all_files = glob.glob(os.path.join(wd,'*{}-SE.MATS.JunctionCountOnly.txt'.format(cell)))
        progress = tnrange(len(all_files))
        for f in all_files:
            basename = os.path.basename(f).replace('.MATS.JunctionCountOnly','')
            df = pd.read_table(f)
            all_junction_counts_dict[basename] = rmats.filter_rmats_df(df)
            progress.update(1)
    return all_junction_counts_dict

wd = '/projects/ps-yeolab3/bay001/maps/current_normed_annotations/se/'
all_junction_counts_dict = get_all_junction_counts_files(wd)

In [159]:
all_junction_counts_dict.keys()

['SUPV3L1-BGHLV17-HepG2-SE.txt',
 'PTBP1-LV11-K562-SE.txt',
 'SRSF3-BGHLV35-HepG2-SE.txt',
 'FASTKD2-BGKLV13-K562-SE.txt',
 'PSIP1-BGKLV21-K562-SE.txt',
 'ZRANB2-BGHLV14-HepG2-SE.txt',
 'ASCC1-LV08-K562-SE.txt',
 'IGF2BP3-LV11-K562-SE.txt',
 'TRIP6-BGKLV29-K562-SE.txt',
 'DDX24-BGHLV14-HepG2-SE.txt',
 'STIP1-BGHLV22-HepG2-SE.txt',
 'METAP2-BGKLV36-K562-SE.txt',
 'CIRBP-BGHLV12-HepG2-SE.txt',
 'SLBP-BGKLV24-K562-SE.txt',
 'GRWD1-BGKLV21-K562-SE.txt',
 'U2AF1-BGHLV30-HepG2-SE.txt',
 'BUD13-BGHLV30-HepG2-SE.txt',
 'TARDBP-BGHLV35-HepG2-SE.txt',
 'NFX1-BGKLV13-K562-SE.txt',
 'FKBP4-BGKLV21-K562-SE.txt',
 'FXR1-BGHLV12-HepG2-SE.txt',
 'DROSHA-BGHcLV05-HepG2-SE.txt',
 'PCBP1-LV11-K562-SE.txt',
 'CKAP4-BGHLV31-HepG2-SE.txt',
 'ABCF1-BGHLV30-HepG2-SE.txt',
 'FMR1-BGKLV36-K562-SE.txt',
 'CCDC124-BGKLV32-K562-SE.txt',
 'ASCC1-BGHLV12-HepG2-SE.txt',
 'PUS1-BGKLV32-K562-SE.txt',
 'RCC2-BGKLV21-K562-SE.txt',
 'HNRNPM-BGKLV29-K562-SE.txt',
 'UTP3-BGHLV33-HepG2-SE.txt',
 'FTO-BGKLV36-K562-SE.txt',
 '

In [171]:
def return_json_id_from_jxc_filename(jxc_filename):
    jxc_filename = jxc_filename.replace('-SE.txt','')
    rbp_name, _, rbp_cell = jxc_filename.split('-')
    return "{}_{}_01".format(rbp_name, rbp_cell) # we don't care about replicates; rmats is one file per 2 reps

def jxc_filename_to_qcat_elements(jxc_filename, qcat_dict):
    return qcat_dict[
        return_json_id_from_jxc_filename(jxc_filename)
    ]

In [172]:
jxc_filename_to_qcat_elements('U2AF1-BGHLV30-HepG2-SE.txt', qcat_dict)

[79, '#4FC601']

In [ ]:
merged